# Testing fastCPH

In [1]:
import numpy

In [2]:
import sklearn

In [3]:
import survwrap

In [4]:
#X, y = survwrap.load_test_data()
#X.shape, y.shape

In [5]:
my_data='flchain'
my_data_df=survwrap.datasets.get_data(my_data)
seed=2312

In [6]:
X, y= my_data_df.get_X_y()
X.shape, y.shape

((6524, 8), (6524,))

### Generate a (stratified) train-test split and Scale the features (only) 

First do the stratified splitting THEN do scaling, parameterized on X_train set ONLY 

In [7]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [8]:
X_train, X_test, y_train, y_test = survwrap.survival_train_test_split(X, y,rng_seed=2311)

In [9]:
scaler = StandardScaler().fit(X_train)
[X_train, X_test] = [ scaler.transform(_) for _ in  [X_train, X_test] ]
X_train.shape, X_test.shape

((4893, 8), (1631, 8))

In [10]:
#X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

balanced partitioning OK. Robst scaler damages the performance of DSM A LOT.
maybe did something wrong. It is standard scaler for now.

In [11]:
survwrap.get_indicator(y).sum(), survwrap.get_indicator(y_train).sum(), survwrap.get_indicator(y_test).sum(),


(1962, 1472, 490)

In [12]:
splitter = survwrap.survival_crossval_splitter(X_train,y_train,n_splits=3, n_repeats=2,rng_seed=2309)
print([ (survwrap.get_indicator(y_train[_[1]]).sum()) for _ in splitter])

[490, 491, 491, 490, 491, 491]


## check possible dimensionality reduction

In [13]:
from sklearn.decomposition import PCA

In [14]:
pca= PCA(n_components=0.995, random_state=2308).fit(X_train)
pca.n_components_

8

Only a modest dimensionality reduction is possible using PCA

In [15]:
## Stratified CV spliter for survival analysis

In [16]:
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold

In [17]:
#fastcph 
import lassonet
from lassonet import LassoNetCoxRegressor

In [18]:
fastcph=LassoNetCoxRegressor(tie_approximation='breslow',
                                hidden_dims=(8,8),
    lambda_start=0.001,
    path_multiplier=1.02,
    backtrack=True,
    #gamma=0.5,
    #val_size=0.15,                        
    verbose=2,
    random_state=seed,
    torch_seed=seed,
    )
fastcph

LassoNetCoxRegressor(backtrack=True, device=device(type='cpu'),
                     hidden_dims=(8, 8), lambda_start=0.001, random_state=2312,
                     tie_approximation='breslow', torch_seed=2312, val_size=0,
                     verbose=2)

In [19]:
#X_train[:5]

In [20]:
#y_swap
#from pandas import DataFrame
#y_df= DataFrame({'time': survwrap.get_time(y_train),'event':survwrap.get_indicator(y_train)})
yup=numpy.column_stack((survwrap.get_time(y_train).astype('float32'),
                        survwrap.get_indicator(y_train).astype('float32')))
print(type(yup))
#yup=numpy.array(list(zip(survwrap.get_indicator(y_train).astype(float),survwrap.get_time(y_train).astype(float))))
yup[:10]
X_train.shape, yup.shape
#numpy.ndarray(y_train[1], y_train[0])

<class 'numpy.ndarray'>


((4893, 8), (4893, 2))

In [21]:
fitted_fastcph = fastcph.fit(X_train,yup)
fitted_fastcph

epoch: 0
loss: 8.509246826171875
epoch: 1
loss: 8.501243591308594
epoch: 2
loss: 8.493334770202637
epoch: 3
loss: 8.485517501831055
epoch: 4
loss: 8.477792739868164
epoch: 5
loss: 8.470152854919434
epoch: 6
loss: 8.462599754333496
epoch: 7
loss: 8.4551362991333
epoch: 8
loss: 8.447758674621582
epoch: 9
loss: 8.440472602844238
epoch: 10
loss: 8.433272361755371
epoch: 11
loss: 8.426156044006348
epoch: 12
loss: 8.419123649597168
epoch: 13
loss: 8.41217041015625
epoch: 14
loss: 8.40530014038086
epoch: 15
loss: 8.398512840270996
epoch: 16
loss: 8.391807556152344
epoch: 17
loss: 8.385175704956055
epoch: 18
loss: 8.378621101379395
epoch: 19
loss: 8.372139930725098
epoch: 20
loss: 8.365907669067383
epoch: 21
loss: 8.360102653503418
epoch: 22
loss: 8.354352951049805
epoch: 23
loss: 8.348657608032227
epoch: 24
loss: 8.34302043914795
epoch: 25
loss: 8.337440490722656
epoch: 26
loss: 8.331913948059082
epoch: 27
loss: 8.326433181762695
epoch: 28
loss: 8.321002960205078
epoch: 29
loss: 8.31561660766

LassoNetCoxRegressor(backtrack=True, device=device(type='cpu'),
                     hidden_dims=(8, 8), lambda_start=0.001, random_state=2312,
                     tie_approximation='breslow', torch_seed=2312, val_size=0,
                     verbose=2)

In [22]:
assert fastcph == fitted_fastcph

In [23]:
fastcph.set_params()

LassoNetCoxRegressor(backtrack=True, device=device(type='cpu'),
                     hidden_dims=(8, 8), lambda_start=0.001, random_state=2312,
                     tie_approximation='breslow', torch_seed=2312, val_size=0,
                     verbose=2)

In [24]:
# MACHECAZZO!
#fastcph.predict(X_train)[:10], fastcph.predict(X_test)[:10]
#lassonet.plot_path(fastcph, fastcph.path, X_train, yup)

In [25]:
fastcph.path_[1].objective

7.556235576868057

In [26]:
#fitted_fastcph.path_.loss , fitted_fastcph.path_.lambda 

lambda_min = min(fitted_fastcph.path_, key=(lambda x: x.objective))
#lambda_min = min([(_.objective, _.lambda_)  for _ in fitted_fastcph.path_])
lambda_min 


HistoryItem(lambda_=0.001999889552662456, state_dict=None, objective=7.554867652909718, loss=7.551932334899902, val_objective=7.554862884538136, val_loss=7.55192756652832, regularization=1.4677400588989258, l2_regularization=tensor(6.6267), l2_regularization_skip=tensor(0.5985), selected=tensor([True, True, True, True, True, True, True, True]), n_iters=10)

In [27]:
fastcph.set_params(lambda_seq=[lambda_min.lambda_],
                   random_state=seed,torch_seed=seed,
                   verbose=1)
refit_fastcph=fastcph.fit(X_train,yup)

epoch: 0
loss: 8.509246826171875
epoch: 1
loss: 8.501243591308594
epoch: 2
loss: 8.493334770202637
epoch: 3
loss: 8.485517501831055
epoch: 4
loss: 8.477792739868164
epoch: 5
loss: 8.470152854919434
epoch: 6
loss: 8.462599754333496
epoch: 7
loss: 8.4551362991333
epoch: 8
loss: 8.447758674621582
epoch: 9
loss: 8.440472602844238
epoch: 10
loss: 8.433272361755371
epoch: 11
loss: 8.426156044006348
epoch: 12
loss: 8.419123649597168
epoch: 13
loss: 8.41217041015625
epoch: 14
loss: 8.40530014038086
epoch: 15
loss: 8.398512840270996
epoch: 16
loss: 8.391807556152344
epoch: 17
loss: 8.385175704956055
epoch: 18
loss: 8.378621101379395
epoch: 19
loss: 8.372139930725098
epoch: 20
loss: 8.365907669067383
epoch: 21
loss: 8.360102653503418
epoch: 22
loss: 8.354352951049805
epoch: 23
loss: 8.348657608032227
epoch: 24
loss: 8.34302043914795
epoch: 25
loss: 8.337440490722656
epoch: 26
loss: 8.331913948059082
epoch: 27
loss: 8.326433181762695
epoch: 28
loss: 8.321002960205078
epoch: 29
loss: 8.31561660766

In [28]:
fastcph.predict(X_train), yup

(array([[0.82628715],
        [0.27492297],
        [3.2277865 ],
        ...,
        [1.584062  ],
        [3.1775737 ],
        [1.2817144 ]], dtype=float32),
 array([[4.680e+03, 0.000e+00],
        [3.459e+03, 0.000e+00],
        [7.710e+02, 1.000e+00],
        ...,
        [4.977e+03, 0.000e+00],
        [6.660e+02, 1.000e+00],
        [4.950e+03, 0.000e+00]], dtype=float32))

In [29]:
fastcph.score(X_train,yup)

0.7971334581495467

In [30]:
#y_swap
#from pandas import DataFrame
#y_df= DataFrame({'time': survwrap.get_time(y_train),'event':survwrap.get_indicator(y_train)})
yup_test=numpy.column_stack((survwrap.get_time(y_test).astype('float32'),
                        survwrap.get_indicator(y_test).astype('float32')))
print(type(yup_test))
X_test.shape, yup_test.shape
#numpy.ndarray(y_train[1], y_train[0])

<class 'numpy.ndarray'>


((1631, 8), (1631, 2))

In [31]:
fastcph.score(X_test,yup_test)

0.7860323741294167

## Test with CV-based lambda selection

In [55]:
cv_set=survwrap.survival_crossval_splitter(X_train,y_train,
                                           n_splits=3,
                                           n_repeats=1,
                                           rng_seed=seed)
cv_set

<generator object _RepeatedSplits.split at 0x7fa9819c6660>

In [36]:
fastcph.get_params(deep=True)

{'M': 10,
 'backtrack': True,
 'batch_size': None,
 'class_weight': None,
 'device': device(type='cpu'),
 'dropout': 0,
 'gamma': 0.0,
 'gamma_skip': 0.0,
 'groups': None,
 'hidden_dims': (8, 8),
 'lambda_seq': [0.001999889552662456],
 'lambda_start': 0.001,
 'n_iters': (1000, 100),
 'optim': None,
 'path_multiplier': 1.02,
 'patience': (100, 10),
 'random_state': 2312,
 'tie_approximation': 'breslow',
 'tol': 0.99,
 'torch_seed': 2312,
 'val_size': 0,
 'verbose': 1}

In [56]:
fastcph_cv=lassonet.LassoNetCoxRegressorCV(cv=cv_set,
                                          hidden_dims=(8,8),
                                          tie_approximation='breslow',
                                          path_multiplier=1.02,
                                           backtrack=True,
                                          torch_seed=seed,random_state=seed)
fastcph_cv

LassoNetCoxRegressorCV(cv=_CVIterableWrapper(cv=[(array([   0,    1, ..., 4889, 4891]), array([   2,    5, ..., 4890, 4892])), (array([   2,    5, ..., 4891, 4892]), array([   0,    1, ..., 4881, 4887])), (array([   0,    1, ..., 4890, 4892]), array([   7,    9, ..., 4889, 4891]))]))

In [57]:
fastcph_cv.fit(X_train,yup)

Choosing lambda with cross-validation:   0%|                                                                                                     | 0/3 [00:00<?, ?it/s]

epoch: 0
loss: 8.097901344299316
epoch: 1
loss: 8.089906692504883
epoch: 2
loss: 8.082008361816406
epoch: 3
loss: 8.07420539855957
epoch: 4
loss: 8.066502571105957
epoch: 5
loss: 8.05888557434082
epoch: 6
loss: 8.051355361938477
epoch: 7
loss: 8.043919563293457
epoch: 8
loss: 8.036574363708496
epoch: 9
loss: 8.029322624206543
epoch: 10
loss: 8.022157669067383
epoch: 11
loss: 8.0150728225708
epoch: 12
loss: 8.008074760437012
epoch: 13
loss: 8.001153945922852
epoch: 14
loss: 7.994317531585693
epoch: 15
loss: 7.987558841705322
epoch: 16
loss: 7.980884075164795
epoch: 17
loss: 7.97428035736084
epoch: 18
loss: 7.96774435043335
epoch: 19
loss: 7.96127986907959
epoch: 20
loss: 7.95506477355957
epoch: 21
loss: 7.949262619018555
epoch: 22
loss: 7.943518161773682
epoch: 23
loss: 7.937826156616211
epoch: 24
loss: 7.932192325592041
epoch: 25
loss: 7.92661714553833
epoch: 26
loss: 7.921092510223389
epoch: 27
loss: 7.915617942810059
epoch: 28
loss: 7.910195350646973
epoch: 29
loss: 7.904819965362549

Choosing lambda with cross-validation:  33%|███████████████████████████████                                                              | 1/3 [00:36<01:12, 36.41s/it]

epoch: 0
loss: 8.12120532989502
epoch: 1
loss: 8.1132173538208
epoch: 2
loss: 8.105315208435059
epoch: 3
loss: 8.097503662109375
epoch: 4
loss: 8.089776039123535
epoch: 5
loss: 8.082139015197754
epoch: 6
loss: 8.07458209991455
epoch: 7
loss: 8.067112922668457
epoch: 8
loss: 8.059725761413574
epoch: 9
loss: 8.0524320602417
epoch: 10
loss: 8.04521656036377
epoch: 11
loss: 8.038080215454102
epoch: 12
loss: 8.031025886535645
epoch: 13
loss: 8.024059295654297
epoch: 14
loss: 8.017178535461426
epoch: 15
loss: 8.010383605957031
epoch: 16
loss: 8.003670692443848
epoch: 17
loss: 7.997034549713135
epoch: 18
loss: 7.990481853485107
epoch: 19
loss: 7.983994483947754
epoch: 20
loss: 7.977760314941406
epoch: 21
loss: 7.971953392028809
epoch: 22
loss: 7.9662065505981445
epoch: 23
loss: 7.960509300231934
epoch: 24
loss: 7.954865455627441
epoch: 25
loss: 7.949283599853516
epoch: 26
loss: 7.943748950958252
epoch: 27
loss: 7.938255786895752
epoch: 28
loss: 7.932799339294434
epoch: 29
loss: 7.927383422851

Choosing lambda with cross-validation:  67%|██████████████████████████████████████████████████████████████                               | 2/3 [01:12<00:36, 36.10s/it]

epoch: 11
loss: 7.758909225463867
epoch: 0
loss: 7.773167133331299
epoch: 1
loss: 7.779519081115723
epoch: 2
loss: 7.784827709197998
epoch: 3
loss: 7.788880348205566
epoch: 4
loss: 7.791557788848877
epoch: 5
loss: 7.792834758758545
epoch: 6
loss: 7.792763710021973
epoch: 7
loss: 7.791479110717773
epoch: 8
loss: 7.7891669273376465
epoch: 9
loss: 7.786057472229004
epoch: 10
loss: 7.782400608062744
epoch: 11
loss: 7.778451442718506
epoch: 0
loss: 7.792834758758545
epoch: 1
loss: 7.7995171546936035
epoch: 2
loss: 7.8051252365112305
epoch: 3
loss: 7.809437274932861
epoch: 4
loss: 7.812325954437256
epoch: 5
loss: 7.81376314163208
epoch: 6
loss: 7.813801288604736
epoch: 7
loss: 7.812575340270996
epoch: 8
loss: 7.810277462005615
epoch: 9
loss: 7.807141304016113
epoch: 10
loss: 7.803421497344971
epoch: 11
loss: 7.799384593963623
epoch: 0
loss: 7.813801288604736
epoch: 1
loss: 7.820809841156006
epoch: 2
loss: 7.82614278793335
epoch: 3
loss: 7.826141834259033
epoch: 4
loss: 7.826143264770508
epoc

Choosing lambda with cross-validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:49<00:00, 36.40s/it]

epoch: 6
loss: 7.82008695602417
epoch: 7
loss: 7.820085525512695
epoch: 8
loss: 7.8200860023498535
epoch: 9
loss: 7.819430351257324
epoch: 10
loss: 7.818080425262451
epoch: 11
loss: 7.8161773681640625
epoch: 0
loss: 8.509246826171875
epoch: 1
loss: 8.501243591308594
epoch: 2
loss: 8.493334770202637
epoch: 3
loss: 8.485517501831055
epoch: 4
loss: 8.477792739868164
epoch: 5
loss: 8.470152854919434
epoch: 6
loss: 8.462599754333496
epoch: 7
loss: 8.4551362991333
epoch: 8
loss: 8.447758674621582
epoch: 9
loss: 8.440472602844238
epoch: 10
loss: 8.433272361755371
epoch: 11
loss: 8.426156044006348
epoch: 12
loss: 8.419123649597168
epoch: 13
loss: 8.41217041015625
epoch: 14
loss: 8.40530014038086
epoch: 15
loss: 8.398512840270996
epoch: 16
loss: 8.391807556152344
epoch: 17
loss: 8.385175704956055
epoch: 18
loss: 8.378621101379395
epoch: 19
loss: 8.372139930725098
epoch: 20
loss: 8.365907669067383
epoch: 21
loss: 8.360102653503418
epoch: 22
loss: 8.354352951049805
epoch: 23
loss: 8.3486576080322

epoch: 31
loss: 8.30496883392334
epoch: 32
loss: 8.299703598022461
epoch: 33
loss: 8.294475555419922
epoch: 34
loss: 8.28927993774414
epoch: 35
loss: 8.284117698669434
epoch: 36
loss: 8.278985977172852
epoch: 37
loss: 8.273885726928711
epoch: 38
loss: 8.268811225891113
epoch: 39
loss: 8.263763427734375
epoch: 40
loss: 8.258737564086914
epoch: 41
loss: 8.253737449645996
epoch: 42
loss: 8.248760223388672
epoch: 43
loss: 8.243805885314941
epoch: 44
loss: 8.238873481750488
epoch: 45
loss: 8.233959197998047
epoch: 46
loss: 8.2290620803833
epoch: 47
loss: 8.2241792678833
epoch: 48
loss: 8.21930980682373
epoch: 49
loss: 8.21445369720459
epoch: 50
loss: 8.209606170654297
epoch: 51
loss: 8.2047700881958
epoch: 52
loss: 8.199942588806152
epoch: 53
loss: 8.195115089416504
epoch: 54
loss: 8.190291404724121
epoch: 55
loss: 8.185471534729004
epoch: 56
loss: 8.180651664733887
epoch: 57
loss: 8.175823211669922
epoch: 58
loss: 8.170991897583008
epoch: 59
loss: 8.16616439819336
epoch: 60
loss: 8.1613416

LassoNetCoxRegressorCV(cv=_CVIterableWrapper(cv=[(array([   0,    1, ..., 4889, 4891]), array([   2,    5, ..., 4890, 4892])), (array([   2,    5, ..., 4891, 4892]), array([   0,    1, ..., 4881, 4887])), (array([   0,    1, ..., 4890, 4892]), array([   7,    9, ..., 4889, 4891]))]))

In [58]:
fastcph_cv.best_cv_scores_, fastcph_cv.best_lambda_, fastcph_cv.best_cv_score_, fastcph_cv.best_selected_


(array([0.78417831, 0.81176678, 0.78648945]),
 31.316526062085696,
 0.7941448451699564,
 tensor([ True,  True,  True,  True,  True,  True, False,  True]))

In [59]:
fastcph_cv.score(X_test,yup_test)

0.7846490143362027

In [60]:
fastcph_cv.path_

[HistoryItem(lambda_=0, state_dict=None, objective=7.554960250854492, loss=7.554960250854492, val_objective=7.554861068725586, val_loss=7.554861068725586, regularization=1.4257103204727173, l2_regularization=tensor(6.5789), l2_regularization_skip=tensor(0.5792), selected=tensor([True, True, True, True, True, True, True, True]), n_iters=299),
 HistoryItem(lambda_=0.2048, state_dict=None, objective=7.844290613555908, loss=7.554886341094971, val_objective=7.844290613555908, val_loss=7.554886341094971, regularization=1.4131067991256714, l2_regularization=tensor(6.5799), l2_regularization_skip=tensor(0.5750), selected=tensor([True, True, True, True, True, True, True, True]), n_iters=10),
 HistoryItem(lambda_=0.20889600000000003, state_dict=None, objective=7.84797142376709, loss=7.554862022399902, val_objective=7.847966655395508, val_loss=7.55485725402832, regularization=1.4031355381011963, l2_regularization=tensor(6.5808), l2_regularization_skip=tensor(0.5717), selected=tensor([True, True, 

In [62]:
#fitted_fastcph.path_.loss , fitted_fastcph.path_.lambda 

lambda_cv_min = min(fastcph_cv.path_, key=(lambda x: x.objective))
#lambda_min = min([(_.objective, _.lambda_)  for _ in fitted_fastcph.path_])
lambda_cv_min


HistoryItem(lambda_=0, state_dict=None, objective=7.554960250854492, loss=7.554960250854492, val_objective=7.554861068725586, val_loss=7.554861068725586, regularization=1.4257103204727173, l2_regularization=tensor(6.5789), l2_regularization_skip=tensor(0.5792), selected=tensor([True, True, True, True, True, True, True, True]), n_iters=299)

In [64]:
fastcph.set_params(lambda_seq=[lambda_cv_min.lambda_],
                   random_state=seed,torch_seed=seed,
                   verbose=1)
refit_fastcph_cv=fastcph.fit(X_train,yup)

epoch: 0
loss: 8.509246826171875
epoch: 1
loss: 8.501243591308594
epoch: 2
loss: 8.493334770202637
epoch: 3
loss: 8.485517501831055
epoch: 4
loss: 8.477792739868164
epoch: 5
loss: 8.470152854919434
epoch: 6
loss: 8.462599754333496
epoch: 7
loss: 8.4551362991333
epoch: 8
loss: 8.447758674621582
epoch: 9
loss: 8.440472602844238
epoch: 10
loss: 8.433272361755371
epoch: 11
loss: 8.426156044006348
epoch: 12
loss: 8.419123649597168
epoch: 13
loss: 8.41217041015625
epoch: 14
loss: 8.40530014038086
epoch: 15
loss: 8.398512840270996
epoch: 16
loss: 8.391807556152344
epoch: 17
loss: 8.385175704956055
epoch: 18
loss: 8.378621101379395
epoch: 19
loss: 8.372139930725098
epoch: 20
loss: 8.365907669067383
epoch: 21
loss: 8.360102653503418
epoch: 22
loss: 8.354352951049805
epoch: 23
loss: 8.348657608032227
epoch: 24
loss: 8.34302043914795
epoch: 25
loss: 8.337440490722656
epoch: 26
loss: 8.331913948059082
epoch: 27
loss: 8.326433181762695
epoch: 28
loss: 8.321002960205078
epoch: 29
loss: 8.31561660766

In [65]:
refit_fastcph_cv.predict(X_test)

array([[ 0.9172462 ],
       [-0.15957029],
       [ 1.9337854 ],
       ...,
       [ 0.5273938 ],
       [ 0.3698055 ],
       [-0.33729953]], dtype=float32)

In [66]:
refit_fastcph_cv.score(X_test,yup_test)

0.7860307785587209